In [9]:
# !pip install darts
# !pip install -U kaleido

## **Step 1: Import all the libraries and modules**

In [16]:
import pandas as pd
import numpy as np
import os
import sys

%load_ext autoreload
%autoreload 2

target_dir = os.path.abspath(os.path.join(os.getcwd(), '..' ))
sys.path.append(target_dir)
# from Models.ARIMAModel.preprocessing import preprocess_df_to_ts

from Models.ARIMAModel.preprocessing import preprocess_df_to_ts
from Models.ARIMAModel.plot import plot_auto_correlation_plot
from Models.ARIMAModel.plot import plot_partial_auto_correlation_plot
from Models.ARIMAModel.evaluation import load_model_sarima_model
from Models.ARIMAModel.evaluation import evaluation_of_model_sarima_Model
from Models.ARIMAModel.test import inspect_seasonality
from Models.ARIMAModel.ARIMA import train_arima_model



from sklearn.model_selection import  train_test_split



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Step 2 : Reading Data Daily Traffic Accident Forecasting**

In [32]:
daily_traffic_accident_data_path = '../data/processed/daily_data.csv'
daily_accident_df = pd.read_csv(daily_traffic_accident_data_path)
daily_accident_df

,daily,daily_accident
0,2018-01-01,234
1,2018-01-02,408
2,2018-01-03,231
3,2018-01-04,225
4,2018-01-05,342
...,...,...
1821,2022-12-27,183
1822,2022-12-28,177
1823,2022-12-29,191
1824,2022-12-30,193


## **Step 3 : Split the Data into train and test data**

In [5]:
processed_train_df, processed_test_df = train_test_split(
        daily_accident_df, test_size=0.245, random_state=42, shuffle=False
    )

## **Step 4 : Preprocess the data into Time series object**

In [6]:
print('[INFO] Converting data into times series types')
ts_train = preprocess_df_to_ts(processed_train_df,'daily','daily_accident','1D')
ts_test = preprocess_df_to_ts(processed_test_df,'daily','daily_accident','1D')

[INFO] Converting data into times series types


## **Step 5 : Plot the Auto Correlation Plot**

In [9]:
print('[INFO] Plotting the auto correlation plot')
plot_auto_correlation_plot(ts_train, m=2, max_lag=30,saving_path='../Datasets_visualization')


[INFO] Plotting the auto correlation plot


## **Step 6 : Plot the Partial Auto Correlation Plot**

In [10]:
print('[INFO] Plotting the Partial auto correlation plot')
plot_partial_auto_correlation_plot(ts_train, m=2, max_lag=30,saving_path='../Datasets_visualization')


[INFO] Plotting the Partial auto correlation plot


## **Step 7 : Checkout the seasonality of training dataset**

In [12]:
print('[INFO] Seasonality in the training datasets')
inspect_seasonality(ts_train)

There is seasonality of order 7.
There is seasonality of order 14.
There is seasonality of order 18.
There is seasonality of order 21.
There is seasonality of order : [7, 14, 18, 21]


[7, 14, 18, 21]

## **Step 8 : Checkout the seasonality of testing dataset**

In [14]:
print('[INFO] Seasonality in the Testing datasets')
inspect_seasonality(ts_test)

[INFO] Seasonality in the Testing datasets
There is seasonality of order 3.
There is seasonality of order 7.
There is seasonality of order 14.
There is seasonality of order : [3, 7, 14]


[3, 7, 14]

## **Step 9 : Training SARIMA Model**

In [24]:
os.makedirs('../pkl',exist_ok=True)
trained_model_file_name = f'../pkl/SARIMAModel_param_p_{p}_d_{d}_q_{q}_P_{P}_D_{D}_Q_{Q}_S_{S}.pkl'
p, d, q, P, D, Q, S = 1,1,1,1,1,1,1
arima_model = train_arima_model(ts_train, p, d, q, P, D, Q, 2)
arima_model.save(trained_model_file_name)

## **Step 10 : Loading the Trainined Model**

In [26]:
sarima_model = load_model_sarima_model(model_name=trained_model_file_name)


## **Step 11 : Evaluation of the SARIMA Model**

In [30]:
evaluation_sarima_df = evaluation_of_model_sarima_Model(trained_model_file_name,
                                                    ts_test=ts_test,
                                                    ts_train=ts_train)

Evaluation of input window : 30 & Horizon : 418
metrics: {'MAE': 533.17, 'RMSE': 609.77, 'MAPE': 187.3, 'MSE': 371816.02}
Evaluation of input window : 60 & Horizon : 388
metrics: {'MAE': 45.35, 'RMSE': 57.79, 'MAPE': 16.66, 'MSE': 3339.45}
Evaluation of input window : 90 & Horizon : 358
metrics: {'MAE': 152.73, 'RMSE': 173.4, 'MAPE': 51.33, 'MSE': 30067.37}
Evaluation of input window : 120 & Horizon : 328
metrics: {'MAE': 60.68, 'RMSE': 74.87, 'MAPE': 19.85, 'MSE': 5605.57}
Evaluation of input window : 150 & Horizon : 298
metrics: {'MAE': 73.29, 'RMSE': 87.85, 'MAPE': 23.51, 'MSE': 7718.02}


## **Step 12 : Saving the Statistical Results**

In [31]:
statistical_result_save_path = os.path.join("../StatisticalResults")
os.makedirs(statistical_result_save_path,exist_ok=True)
evaluation_sarima_df.to_csv(
    f"{statistical_result_save_path}/sarima_evaluation_results.csv", index=False
)
